In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import re
import numpy as np
import os
from collections import defaultdict
from shutil import copy
from shutil import copytree, rmtree
import matplotlib.pyplot as plt

In [ ]:
def correct_nested_strings(x):
    import re
    import numpy as np
    x = re.sub(r'\W*(na)*\W*(na)+\W*',' ',x).strip()
    if x != '':
        if (len(x.split(" ")) ==1) & (re.match(r',',x)==False):
            return x
        else:
            x = re.sub(r',', " ", x)
            d = x.split(" ")
            d = np.array(d)
        
            d = np.unique(d).tolist()
            return ','.join(d)
    else: 
        return None

file_lst = []
for i in range(1,7):
    i = str(i)
    file_name = "cal_data_"+ i +'.csv'
    f = pd.read_csv(file_name)
    f = f.groupby(['title', 'recipe_link', 'img_link', 'ingredients', 'yield', 'nutrients',
       'total_calories', 'total_protein',
       'total_fat', 'total_carb']).agg({'meal_type': lambda x: ",".join(x), 'subregion':  lambda x: ",".join(x), 'region':  lambda x: ",".join(x)}).reset_index()
    f[['region', 'subregion', 'meal_type']] = f[['region', 'subregion', 'meal_type']].applymap(correct_nested_strings)
    file_lst.append(f)
new_df = pd.concat(file_lst).reset_index(drop=True)
new_df.to_csv('recipe_data_nutrition.csv')

In [33]:
cal_data = new_df.copy()
cal_data = cal_data.drop(cal_data.iloc[:,0:1], axis =1)
cal_data.meal_type = cal_data.meal_type.str.split(',')
cal_data = cal_data.explode('meal_type')
cal_data = cal_data.fillna('Unknown')
cal_data.meal_type = cal_data.meal_type.apply(lambda x: "Unknown" if x  == "" else x)
print(cal_data[['meal_type']].value_counts().reset_index().rename({0: 'count'}, axis = 1))

         meal_type  count
0          Unknown   3427
1             main   1926
2          dessert    853
3             side    568
4  soups_and_stews    524
5        appetizer    499
6            salad    469
7            bread    348
8           drinks    148


In [22]:
cal_data.meal_type = cal_data.meal_type.str.split(',')
cal_data = cal_data.explode('meal_type')


,title,recipe_link,img_link,ingredients,yield,nutrients,total_calories,total_protein,total_fat,total_carb,meal_type,subregion,region
219,Chicken Tortilla Soup,https://www.allrecipes.com/recipe/13351/chicke...,https://www.allrecipes.com/thmb/ZfHQdy0fyzLeG3...,"['1 tablespoon olive oil', '1 medium onion, ch...",8 servings,"{'calories': '363 kcal', 'carbohydrateContent'...",2492.93,151.21,110.64,258.05,main,mexican,latin_american
219,Chicken Tortilla Soup,https://www.allrecipes.com/recipe/13351/chicke...,https://www.allrecipes.com/thmb/ZfHQdy0fyzLeG3...,"['1 tablespoon olive oil', '1 medium onion, ch...",8 servings,"{'calories': '363 kcal', 'carbohydrateContent'...",2492.93,151.21,110.64,258.05,soups_and_stews,mexican,latin_american
2565,Chicken Tortilla Soup,https://www.allrecipes.com/recipe/13367/chicke...,https://www.allrecipes.com/thmb/3UJZqwJK1rp8w1...,"['6 tablespoons vegetable oil', '8 (6 inch) co...",6 servings,"{'calories': '418 kcal', 'carbohydrateContent'...",2493.32,219.13,117.94,146.98,soups_and_stews,jewish,usa


In [3]:
def clean_data(data):
    df = data.copy()
    # change col names
    df=df.rename(columns={'yield': 'servings'})
    # get scapped nutirents
    df['nutrients'] = df['nutrients'].apply(lambda x: x.replace("'", '"'))
    temp_df = pd.json_normalize(df.nutrients.apply(json.loads))
    temp_df = temp_df.loc[:,['calories','carbohydrateContent', 'proteinContent','fatContent']]
    names = ['scrapped_'+i for i in temp_df.columns.tolist()]
    names_dict = dict(zip(temp_df.columns.tolist(), names))
    temp_df = temp_df.rename(columns = names_dict)

    # get only values
    for i in temp_df.columns:
        temp_df[i] = temp_df[i].apply(lambda x: x.split(" ")[0] if isinstance(x, str) ==True else x)
        temp_df[i] = temp_df[i].astype('float')
    # concat back to all data
    df = df.drop('nutrients', axis =1)
    df = pd.concat([df, temp_df], axis =1)
    
    # get num servings
    df['servings'] = df['servings'].apply(lambda x: re.search(r'\b[\d]+\b',x).group(0))
    
        # adjust amount to per servings 
    cols = ['total_calories','total_protein', 'total_fat', 'total_carb']
    new_names = ['calories_per_serving','protein_per_serving','fat_per_serving','carb_per_serving']
    for i in cols:
        df[i] = df[i]/ df['servings'].astype(int)
        df[i] = df[i].apply(lambda x: int(round(x,0)))
    df =df.rename(columns=dict(zip(cols, new_names)))
    
    
    # drop nas
    df = df.dropna(subset=names)
    
    # get cal diff
    df['calories_diff'] = (df['scrapped_calories'] - df['calories_per_serving'])
    return df

In [4]:
clean_cal_data = clean_data(cal_data)
# to get rid of data with high diff
index = clean_cal_data[(clean_cal_data.scrapped_calories*0.80 > clean_cal_data.calories_per_serving) | \
              (clean_cal_data.scrapped_calories*1.2 < clean_cal_data.calories_per_serving)].index
d_2 = clean_cal_data.drop(index)
d_2

,title,recipe_link,img_link,ingredients,servings,calories_per_serving,protein_per_serving,fat_per_serving,carb_per_serving,meal_type,subregion,region,scrapped_calories,scrapped_carbohydrateContent,scrapped_proteinContent,scrapped_fatContent,calories_diff
1,2 Minute Cheese Quesadillas,https://www.allrecipes.com/recipe/228115/2-min...,https://www.allrecipes.com/thmb/GtMGRHlRtWfwSV...,"['2 (10 inch) flour tortillas', '1 cup shredde...",2,388,17,22,29,main,mexican,latin_american,446.0,37.0,20.0,24.0,58.0
3,3 Milk Cake or Tres Leches,https://www.allrecipes.com/recipe/221906/3-mil...,https://www.allrecipes.com/thmb/vpL0cHabng-QAP...,"['6 eggs, separated', '2 cups white sugar', '2...",10,795,16,39,96,dessert,mexican,latin_american,748.0,88.0,15.0,38.0,-47.0
4,3-Cheese Enchiladas,https://www.allrecipes.com/recipe/222009/3-che...,https://www.allrecipes.com/thmb/2qi9QJVdL5NNPi...,"['3 cups salsa, divided', '1 (8 ounce) package...",12,384,17,25,25,main,mexican,latin_american,426.0,33.0,18.0,25.0,42.0
8,Abuela's Picadillo,https://www.allrecipes.com/recipe/255660/abuel...,https://www.allrecipes.com/thmb/Bib6BmBuCKN3Ui...,"['2 pounds lean ground beef', '0.25 cup olive ...",8,381,34,22,9,NaN,carribean,latin_american,378.0,22.0,24.0,22.0,-3.0
9,Addictive Sweet Potato Burritos,https://www.allrecipes.com/recipe/13954/addict...,https://www.allrecipes.com/thmb/KTjaFdpZU4q9oX...,"['1 tablespoon vegetable oil', '1 onion, chopp...",12,427,18,12,63,main,mexican,latin_american,505.0,77.0,20.0,9.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8566,Zucchini Saute,https://www.allrecipes.com/recipe/25267/zucchi...,https://www.allrecipes.com/thmb/prRSN2wSrGs4OY...,"['1 tablespoon olive oil', '0.5 red onion, dic...",6,59,3,3,7,side,"italian,jewish","european,usa",68.0,9.0,3.0,3.0,9.0
8567,Zucchini and Potato Bake,https://www.allrecipes.com/recipe/89195/zucchi...,https://www.allrecipes.com/thmb/88ymyhTcvc2zq8...,"['4 medium potatoes, peeled and cut into large...",6,235,5,10,34,side,jewish,usa,243.0,35.0,5.0,10.0,8.0
8568,Zucchini with Chickpea and Mushroom Stuffing,https://www.allrecipes.com/recipe/25920/zucchi...,https://www.allrecipes.com/thmb/savfWtmsiRFVwi...,"['4 zucchini, halved', '1 tablespoon olive oil...",8,125,6,4,19,main,jewish,usa,107.0,18.0,5.0,3.0,-18.0
8569,Zucchini with Dill Weed and Garlic-Yogurt Sauce,https://www.allrecipes.com/recipe/200196/zucch...,https://www.allrecipes.com/thmb/4S2mv4ef9kDAiD...,"['0.25 cup olive oil', '1 onion, finely choppe...",4,290,10,16,29,NaN,turkish,middle_eastern,292.0,30.0,10.0,16.0,2.0


In [ ]:
# drop them because API interpreted wrong

sns.displot(x=d_2['calories_per_serving']) 

# FOOD101

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models
import cv2

In [3]:
food, info = tfds.load("food101", with_info=True)

In [6]:
food


{'train': <PrefetchDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>,
 'validation': <PrefetchDataset element_spec={'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>}

In [9]:
!python -m wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

C:\ProgramData\Anaconda3\python.exe: No module named wget


In [ ]:
def get_data_extract():
    if "food-101" in os.listdir():
        print("Dataset already exists")
    else:
        print("Downloading the data...")
        !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
        print("Dataset downloaded!")
        print("Extracting data..")
        !tar xzvf food-101.tar.gz
        print("Extraction done!")

In [ ]:
get_data_extract()

In [ ]:
os.listdir('./food-101/images')